In [1]:
import os
from pathlib import Path
import pandas as pd
# import numpy as np
from sqlalchemy import create_engine,text
# from scipy import stats
import plotly.express as px
import toml
import sys

sys.path.append("../../notebook_styling")
import psrc_theme

input_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'input_configuration.toml'))
valid_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'validation_configuration.toml'))
# emme_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration','emme_configuration.toml'))
# network_config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration','network_configuration.toml'))

# table format
pd.options.display.float_format = '{:0,.0f}'.format
format_percent = "{:,.1%}".format

In [2]:
# summary data location
df_result = pd.read_csv(Path(valid_config['model_dir']) / 'outputs/network/network_results.csv')

# observed counts location
conn = create_engine('sqlite:///../../../../inputs/db/'+input_config['db_name'])
df_observed_bike = pd.read_sql(text("SELECT * FROM observed_bike_2023"), con=conn.connect())

In [37]:
df_result[df_result['@countid'] == 2046][['tod','@bvol']].to_clipboard()

In [18]:
df = df_result[['@bvol','@countid']].groupby('@countid').sum()[['@bvol']]

In [20]:
df = df.merge(df_observed_bike, left_on='@countid', right_on='countID', suffixes=('', '_obs'))

In [21]:
df = df.groupby('countID').sum()[['location','@bvol','observed_counts']]

In [29]:
df

,location,@bvol,observed_counts
countID,,,
"2,046",Fremont Bridge,"4,358",2350
"6,002",Spokane St,"1,290",580
"6,003",2nd Ave Cycle Track North of Marion St,"2,585",643
"6,004",Broadway Cycle Track North Of E Union St,"2,097",234
"6,005",Burke Gilman Trail north of NE 70th St,319,679
"6,006",Chief Sealth Trail North of Thistle,156,57
"6,007",MTS Trail west of I-90 Bridge,834,364
"6,022",112th Ave NE,116,46
"6,023",114th Ave SE,358,65


In [34]:
# Create scatter plot of @bvol vs observed_counts
fig = px.scatter(df, x='@bvol', y='observed_counts', text='location',
                 labels={'location': 'location', '@bvol': 'Simulated Bike Volume', 'observed_counts': 'Observed Bike Volume'},
                 title='Simulated vs Observed Bike Volumes',
)
fig.show()